In [1]:
from chat_solution.create_db import create_db

db = create_db()

Loading environment variables from /Users/jean.machado@getyourguide.com/prj/rag-workshop/.env


/Users/jean.machado@getyourguide.com/prj/rag-workshop/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Created 6 chunks of size 700 with overlap 200
Loading database from /tmp/embedding_db.pkl
Database saved to /tmp/embedding_db.pkl
Database saved successfully


In [2]:
print(db.retrieve("How do you pick a green?"))

['rs to when a model generates information that sounds plausible but is factually incorrect or unsupported by the training data.\n\n\n\nPrompt engineering\nPrompt Tuning: Technique of adjusting the embeddings of prompts rather than the prompt text itself to improve model output. It is \n\nWhat is Fine-tunning?\nTBD\n\n What are multimodal models? \nMultimodal models can process and integrate multiple types of data, such as text, images, and audio, to create richer and more comprehensive outputs across different input types.\n\nEvaluation\nEvaluation in LLMs measures model performance using metrics such as accuracy, relevance, and coherence, assessing how well the model fulfills its intended purpose and meet', 'rs to when a model generates information that sounds plausible but is factually incorrect or unsupported by the training data.\n\n\n\nPrompt engineering\nPrompt Tuning: Technique of adjusting the embeddings of prompts rather than the prompt text itself to improve model output. It

In [1]:
# User input and response handling
from chat_solution.rag import QuizRag

query1 = "what is up?"
query2 = "How do you pick a green?"
rag = QuizRag()  
response = rag.query(query2)
print(response)

Loading environment variables from /Users/jean.machado@getyourguide.com/prj/rag-workshop/.env


/Users/jean.machado@getyourguide.com/prj/rag-workshop/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loading database from /tmp/embedding_db.pkl
Found 5 documents, first 500 characters: ['rs to when a model generates information that sounds plausible but is factually incorrect or unsupported by the training data.\n\n\n\nPrompt engineering\nPrompt Tuning: Technique of adjusting the embeddings of prompts rather than the prompt text itself to improve model output. It is \n\nWhat is Fine-tunning?\nTBD\n\n What are multimodal models? \nMultimodal models can process and integrate multiple types of data, such as text, images, and audio, to create richer and more comprehensive outputs across different input types.\n\nEvaluation\nEvaluation in LLMs measures model performance using metrics such as accuracy, relevance, and coherence, assessing how well the model fulfills its intended purpose and meet', 'rs to when a model generates information that sounds plausible but is factually incorrect or unsupported by the training data.\n\n\n\nPrompt engineering\nPrompt Tuning: Technique of adjusting the

In [168]:
# Convert each text chunk to a LangChain Document
from langchain.schema import Document

langchain_docs = [
    Document(page_content=text, metadata={"source": f"chunk_{i+1}"})
    for i, text in enumerate(text_chunks)
]

print(len(langchain_docs))
# Display documents with metadata
#for doc in langchain_docs:
#    print(doc.page_content, doc.metadata)

180


In [143]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_mistralai import ChatMistralAI
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]


In [144]:
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

In [145]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(langchain_docs, testset_size=10)

#testset = generator.generategenerate_with_langchain_docs(langchain_docs[:10], test_size=10, 
#                                                 distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


In [128]:
## Prepared questions dataset from huggingface
from datasets import load_dataset
dataset = load_dataset("atitaarora/food_lab_green_qna", split="train")
len(dataset)

34

In [129]:
## Sample question data
dataset[2]

{'query': '{"question": "If the dark green leaves of chicory are removed and discarded, then how might this affect the overall texture and flavor of the salad?"}',
 'reference_contexts': ['ENDIVE AND CHICORY\nSALAD\nWITH GRAPEFRUIT,\nCRANBERRIES, AND FIG AND\nPUMPKIN SEED VINAIGRETTE\nSERVES 4\n1 head chicory, dark green leaves removed and\ndiscarded, pale white and yellow sections washed, spun\ndry, and torn into 2-inch pieces\n2 Belgian endives, bottoms trimmed, separated into'],
 'reference_answer': '{"answer": "Removing and discarding the dark green leaves of chicory results in a salad with a milder flavor and a more tender texture, as the pale white and yellow sections are less bitter and more delicate."}',
 'new_code': 'true',
 'node_metadata': {'excerpt_keywords': 'Keywords: Endive, Chicory, Salad, Grapefruit, Cranberries, Fig, Pumpkin Seed, Vinaigrette',
  'page_number': 90,
  'source_file_name': '/tmp/food_lab_green_chapter.pdf'},
 'question_type': 'CONDITIONAL',
 'metadata': 

In [ ]:
#from ragas import evaluate
#results = evaluate(dataset=dataset, metrics=metrics, llm=evaluator_llm)

In [155]:
## Preparation of Eval dataset for RAGAS (https://docs.ragas.io/en/stable/concepts/components/eval_sample/?h=singleturnsample#example)
##for ragas dataset needs to be in the designated format 
from ragas import EvaluationDataset, SingleTurnSample
from ragas.metrics import Faithfulness
from datasets import load_dataset
from ragas import evaluate
import time

samples = []
eval_size = 5

for i in range(eval_size):
    entry = dataset[i]
    
    # Perform the query with a delay to limit to 1 request per second
    user_query = entry['query']
    response = rag.query(user_query)
    
    sample = SingleTurnSample(
        user_input=user_query,
        reference=entry['reference_answer'],
        response=response,
        retrieved_contexts=db.retrieve(user_query),
    )
    samples.append(sample)
    
    # Wait for 1-2 second before proceeding to the next iteration as we are limited by Mistral API
    time.sleep(2)


In [157]:
import pandas as pd
df = pd.DataFrame(samples)

# Display the DataFrame as a table
print(df)

                                                   0  \
0  (user_input, {"question": "If all vegetables a...   
1  (user_input, {"question": "How might the combi...   
2  (user_input, {"question": "If the dark green l...   
3  (user_input, {"question": "What type of leaves...   
4  (user_input, {"question": "What ingredient is ...   

                                                   1  \
0  (retrieved_contexts, [ want to use plenty ofwa...   
1  (retrieved_contexts, [a dry cooking method, th...   
2  (retrieved_contexts, [ vigorously before using...   
3  (retrieved_contexts, [omatoes, toasted nuts, a...   
4  (retrieved_contexts, [aseddressings tend to be...   

                            2  \
0  (reference_contexts, None)   
1  (reference_contexts, None)   
2  (reference_contexts, None)   
3  (reference_contexts, None)   
4  (reference_contexts, None)   

                                                   3                        4  \
0  (response, If all vegetables are not the sa

In [163]:
## Actual Evaluation
from ragas.metrics import LLMContextPrecisionWithReference
from ragas.metrics import NonLLMContextRecall
from ragas.metrics import LLMContextRecall
from ragas.metrics import Faithfulness
from ragas.metrics import ResponseRelevancy

eval_dataset = EvaluationDataset(samples=samples)

faithfulness = Faithfulness()
context_precision = LLMContextPrecisionWithReference()
context_recall = NonLLMContextRecall()
llm_context_recall = LLMContextRecall()
answer_relevancy = ResponseRelevancy()

eval_results = evaluate(
        dataset=eval_dataset,
        metrics=[
                faithfulness,
                answer_relevancy,
                #context_recall, #This metric [non_llm_context_recall] that is used requires the following additional columns ['reference_contexts'] to be present in the dataset.
                llm_context_recall,
                context_precision,
        ],
       #llm=evaluator_llm
       raise_exceptions=False 
    )
#eval_results = evaluate(
#    dataset=eval_dataset,
#    metrics=[metric],
#llm=evaluator_llm
#)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

No statements were generated from the answer.


In [164]:
evaluation_result_df = eval_results.to_pandas()
evaluation_result_df.iloc[:5]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,llm_context_precision_with_reference
0,"{""question"": ""If all vegetables are trimmed to...",[ want to use plenty ofwater—it retains its te...,"If all vegetables are not the same type, they ...","{""answer"": ""The natural diversity of vegetable...",0.500000,0.92057,1.0,1.0
1,"{""question"": ""How might the combination of bee...","[a dry cooking method, they barely lose any ju...","""Beets bring a sweet and earthy flavor, while ...","{""answer"": ""The combination of beets, olive oi...",0.636364,0.92550,1.0,1.0
2,"{""question"": ""If the dark green leaves of chic...",[ vigorously before using.KNIFE SKILLS:How to ...,If the dark green leaves of chicory are remove...,"{""answer"": ""Removing and discarding the dark g...",0.500000,0.00000,1.0,0.7
3,"{""question"": ""What type of leaves are not reco...","[omatoes, toasted nuts, and herbs?Probably not...","""It's not my place to tell you that.""","{""answer"": ""Darker green leaves are not recomm...",NaN,0.00000,0.0,0.0
4,"{""question"": ""What ingredient is not recommend...",[aseddressings tend to be thicker and creamier...,It is out of my pay grade.,"{""answer"": ""The shake-it-in-a-jar method is no...",0.000000,0.00000,0.0,0.0
